## 순전파와 역전파
`신경망(Neural Network)`은 어떤 입력 데이터에 대해 실행되는 중첩된 함수들의 집합체입니다. 신경망을 아래 2단계를 거쳐 학습됩니다.
* 순전파(Forward Propagation)
* 역전파(Backward Propagation)

![](./Static/img1.png)

### 순전파(Forward Propagation)
순전파 단계에서, 신경망은 정답을 맞추기 위해 최선의 추측(best guess)을 합니다. 이렇게 추측을 하기 위해서 입력 데이터를 각 함수들에서 실행합니다.

### 역전파(Backward Propagation)
역전파 단계에서, 신경망은 추측한 값에서 발생한 error에 비례하여 파라미터들을 적절히 업데이트합니다. 출력(output)으로부터 역방향으로 이동하면서 오류에 대한 함수들의 매개변수들이 미분값(gradient)를 수집하고, `경사하강법(gradient descent)`을 사용하여 매개변수들을 최적화 합니다.

## 뉴럴네트워크 학습 알고리즘

* 모든 가중치 w를 임의로 생성
    * Forward Propagation

* 입력변수 값과 입력층과 은닉층 사이의 w값을 이용하여 은닉노드의 값을 계산
    * 선형결합 후 activation한 값

* 은닉노드의 값과 은닉층과 출력층 사이의 w값을 이용하여 출력노드의 값을 계산
    * 선형결합 후 activation한 값
    * Backward Propagation

* 계산된 출력노드의 값과 실제 출력변수의 값의 차이를 줄일 수 있도록 은닉층과 출력층 사이의 w값을 업데이트

* 에러가 충분히 줄어들 때까지 반복

## Autograd 개념
`pyTorch`를 이용해 코드를 작성할때 이러한 역전파를 통해 파라미터를 업데이트하는 방법은 바로 `Autograd`입니다. 차근차근 코드를 통해 알아보도록 합시다. Autograd에 대해 알아보기 위해 간단한 MLP(Multi-Layer Perceptron)을 예시로 들겠습니다.

In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

C:\Users\LUDOBICO\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


## Batch size
batch_size는 딥러닝 모델이 `파라미터를 업데이트할 때 계산되는 데이터 묶음의 갯수`입니다. 앞에서 Neural Network가 순전파와 역전파를 수행하면서 파라미터를 업데이트를 한다고 소개 드렸는데, 이러한 업데이트를 수행하는 데 사용되는 데이터 단위(갯수)가 되는 것이 바로 바로 batch_size입니다. 아래 예시에서 batch_size를 32로 지정해줬는데, 이는 코드 작성자 마음대로 정해주는 하이퍼파라미터입니다.

In [2]:
BATCH_SIZE = 32

![](./Static/batch_size.png)

* `INPUT_SIZE`는 딥러닝 모델의 입력값의 크기이며, 입력층의 노드 수를 의미합니다.

* `HIDDEN_SIZE`는 입력값에 다수의 파라미터를 사용하여 계산되는 값의 개수로, 은닉 층의 노드 수를 의미합니다.

* `OUTPUT_SIZE`는 은닉값에 다수의 파라미터를 사용하여 계산되는 결과값의 개수로, 출력 층의 노드 수를 의미합니다.

* `LEARNING_RATE`는 Gradient를 업데이트할 때 곱해주는 0과 1 사이에 존재하는 값입니다. 좀 더 느리지만 섬세하고 촘촘히 업데이트를 원하면 작은 rate를, 좀 더 빠르게 업데이트를 원하면 큰 rate를 줄 수 있습니다.

In [64]:
# 하이퍼파라미터 지정
INPUT_SIZE = 1000
HIDDEN_SIZE = 100
OUTPUT_SIZE = 2
LEARNING_RATE = 1e-6

In [65]:
# 임의의 x,y, weight 정의
# x : input 값 >> (32, 1000)

x = torch.randn(BATCH_SIZE, INPUT_SIZE, device=device,
                dtype=torch.float).requires_grad_()

# y : output 값 >> (32, 2)
y = torch.randn(BATCH_SIZE, OUTPUT_SIZE, device=device,
                dtype=torch.float).requires_grad_()

# w1 : input -> hidden >> (1000, 100)
w1 = torch.randn(INPUT_SIZE, HIDDEN_SIZE, device=device,
                 dtype=torch.float).requires_grad_()

# w2 : hidden -> output >> (100, 2)
w2 = torch.randn(HIDDEN_SIZE, OUTPUT_SIZE, device=device,
                 dtype=torch.float).requires_grad_()

# 실험환경을 위해 다음과 같이 임의의 값 input(x), output(y), weight(w1,w2)
# 를 정의해 줍니다.

# 이때, requires_grad=True는 모든 연산들을 추적해야 한다고 알려줍니다.


## Train Model (iteration = 500)

* 본 포스트는 Autograd를 확인해보는 포스트이므로, 단순하게 for문을 이용하여 500번의 iteration을 수행하도록 코드를 작성하였습니다.

* `torch.mm()` : mm은 matrix multiplication의 줄임말으로, `행렬의 곱셈`을 의미합니다.

* `torch.nn.ReLU()` : ReLU함수, ReLU는 max(0,x)를 의미하는 함수인데, 0보다 작아지게 되면 0이 되고, 그 이상은 값을 유지한다는 특성을 가지고 있습니다.

* `loss.backward()` : loss에 대해서 backward를 호출한 것으로, autograd는 각 파라미터 값에 대해 미분값을 계산하고 이를 각 텐서의 grad속성(attribute)에 저장합니다.

* `with torch.no_grad()` : 미분값 계산을 사용하지 않도록 설정하는 컨텍스트-관리자(context-manager)입니다. 해당모드는 입력에 requires_grad = True 가 있어도 False로 바꿔줍니다.

In [70]:
import torch

# 500 iteration
for t in range(1, 10001):
    # 은닉값
    hidden = torch.nn.functional.relu(x.mm(w1))
    
    # 예측값
    y_pred = hidden.mm(w2)
    
    # 오차제곱합 계산
    loss = (y_pred - y).pow(2).sum()
    
    # iteration 100마다 기록
    if t % 100 == 0:
        print(t, 'th Iteration: ', sep= '')
        print('>>>> Loss: ', loss.item())
        
    # Loss의 Gradient 계산
    loss.backward()
    
    # 해당 시점의 Gradient값을 고정
    with torch.no_grad():
        # 가중치 업데이트
        w1 -= LEARNING_RATE * w1.grad
        w2 -= LEARNING_RATE * w2.grad
        
        # 가중치 Gradient 초기화(0)
        w1.grad.zero_()
        w2.grad.zero_()
    

100th Iteration: 
>>>> Loss:  3.5938907405608234e-09
200th Iteration: 
>>>> Loss:  3.728950481729498e-09
300th Iteration: 
>>>> Loss:  2.8433300158781094e-09
400th Iteration: 
>>>> Loss:  2.824742217910625e-09
500th Iteration: 
>>>> Loss:  2.474245919259488e-09
600th Iteration: 
>>>> Loss:  2.4942548026984923e-09
700th Iteration: 
>>>> Loss:  2.793649089838368e-09
800th Iteration: 
>>>> Loss:  3.631009271032326e-09
900th Iteration: 
>>>> Loss:  3.7303715672010185e-09
1000th Iteration: 
>>>> Loss:  3.968147588295778e-09
1100th Iteration: 
>>>> Loss:  3.3321765346983057e-09
1200th Iteration: 
>>>> Loss:  2.9891620290101173e-09
1300th Iteration: 
>>>> Loss:  3.324481134825419e-09
1400th Iteration: 
>>>> Loss:  4.091185168419997e-09
1500th Iteration: 
>>>> Loss:  3.6176228679352107e-09
1600th Iteration: 
>>>> Loss:  3.6694638438916627e-09
1700th Iteration: 
>>>> Loss:  3.1316897963762358e-09
1800th Iteration: 
>>>> Loss:  3.1316897963762358e-09
1900th Iteration: 
>>>> Loss:  3.250265168119